In [4]:
import sys
sys.path.append('/home/cryptobrahman/Own/football_competitions_research/own_modules/') 
sys.path.append('/mnt/KINGSTON_120/Own/football_competitions_research/own_modules/') 

In [5]:
import pandas as pd
import numpy as np
from numpy import arange, isclose
from copy import deepcopy
from pprint import pprint

from flatlib import const
from flatlib import aspects
from flatlib.chart import Chart
from flatlib.datetime import Datetime
from flatlib.geopos import GeoPos
from flatlib.object import GenericObject
from flatlib.tools.chartdynamics import ChartDynamics
from flatlib.tools import arabicparts
from flatlib.dignities import essential

from _astro_constants import AstrologicalConstants, AstrologicalPoints, TransformDoubleValues, ObjectsPrepare, AspectsPrepare, AspectsClearing

pd.set_option('display.max_columns', 200)  # or 1000
pd.set_option('display.max_rows', 200)  # or 1000
pd.set_option('display.max_colwidth', 100)  # or 199

In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
data = {'date':['2015/03/13', '2016/05/23', '2018/10/03'], 'utc_time':['14:00', '12:30', '00:15'], 'lon':['38n32', '08n07', '14s42'], 'lat':['8w54', '18e14','36w04']}
df = pd.DataFrame(data)
df

,date,utc_time,lon,lat
0,2015/03/13,14:00,38n32,8w54
1,2016/05/23,12:30,08n07,18e14
2,2018/10/03,00:15,14s42,36w04


### Get astrological objects

In [8]:
df['charts'] = AstrologicalPoints.charts_calculate(df, 'date', 'utc_time', 'lon', 'lat')

In [9]:
necessary_constants = AstrologicalConstants.necessary_constants
rulers_constants    = AstrologicalConstants.rulers_constants
antes_objects       = AstrologicalConstants.for_antes_objects
pars_constants      = AstrologicalConstants.pars_constants

rulers_constant_col_names = AstrologicalPoints.rulers_col_names(rulers_constants)
rulers_pars_col_names     = AstrologicalPoints.rulers_col_names(pars_constants)
rulers_pars_col_names.append('ruler_antes_pars_fortuna')

for obj in rulers_constants + necessary_constants + pars_constants:
    df[str.lower(obj)] = AstrologicalPoints.calculate_astro_objects(df.charts, obj)

for name, obj in zip(rulers_constant_col_names, rulers_constants):
    obj = str.lower(obj)
    df[name] = AstrologicalPoints.ruler_of_object(df[obj])
    df[name] = AstrologicalPoints.chart_object_attributes(df, 'charts', name)
    
for obj in antes_objects:
    antes_name = 'antes_' + obj
    df[antes_name] = AstrologicalPoints.antes_objects_calc(df, obj)

antes_objects = []

for col in df.columns:
    if col.startswith('antes'):
        AstrologicalPoints.rename_antes_objects(df, col)
        antes_objects.append(col)

pars_constants.append('ANTES_PARS_FORTUNA')

for name, obj in zip(rulers_pars_col_names, pars_constants):
    obj = str.lower(obj)
    df[name] = AstrologicalPoints.ruler_of_object(df[obj])
    df[name] = AstrologicalPoints.chart_object_attributes(df, 'charts', name)

houses_objects = AstrologicalConstants.houses_objects
pars_objects   = AstrologicalConstants.pars_objects
ruler_objects  = AstrologicalConstants.ruler_objects
main_objects   = AstrologicalConstants.main_objects

cols_for_id    = houses_objects + antes_objects + pars_objects + ruler_objects + main_objects 

df['id_for_aspects'] = AstrologicalPoints.id_for_aspekts(df, cols_for_id)
df['transform_id'] = df.id_for_aspects.map(lambda x: TransformDoubleValues(x).conbine_class_methods())
df.drop(columns='id_for_aspects', inplace=True)

In [10]:
df

,date,utc_time,lon,lat,charts,asc,desc,mc,ic,pars_fortuna,sun,moon,saturn,uranus,neptune,pluto,chiron,north_node,south_node,pars_spirit,pars_glory,pars_crest,pars_rock,ruler_asc,ruler_desc,ruler_mc,ruler_ic,ruler_pars_fortuna,antes_moon,antes_ruler_asc,antes_ruler_desc,antes_ruler_mc,antes_ruler_ic,antes_pars_fortuna,antes_uranus,antes_neptune,antes_pluto,antes_chiron,ruler_pars_spirit,ruler_pars_glory,ruler_pars_crest,ruler_pars_rock,ruler_antes_pars_fortuna,transform_id
0,2015/03/13,14:00,38n32,8w54,<flatlib.chart.Chart object at 0x7f1e28eec550>,<Asc Cancer +27:24:38>,<Desc Capricorn +27:24:38>,<MC Aries +13:01:44>,<IC Libra +13:01:44>,<Pars Fortuna Aries +25:28:18 +00:00:00>,<Sun Pisces +22:39:56 +00:59:52>,<Moon Sagittarius +20:43:35 +13:13:02>,<Saturn Sagittarius +04:55:45 +00:00:06>,<Uranus Aries +15:06:33 +00:03:14>,<Neptune Pisces +07:50:38 +00:02:13>,<Pluto Capricorn +15:14:35 +00:01:02>,<Chiron Pisces +17:36:15 +00:03:43>,<North Node Libra +11:08:52 -00:03:11>,<South Node Aries +11:08:52 -00:03:11>,<Pars Spirit Libra +29:20:59 +00:00:00>,<Pars Glory Sagittarius +18:23:55 +00:00:00>,<Pars Crest Sagittarius +08:56:02 +00:00:00>,<Pars Rock Aries +09:40:27 +00:00:00>,<Moon Sagittarius +20:43:35 +13:13:02>,<Saturn Sagittarius +04:55:45 +00:00:06>,<Mars Aries +16:27:08 +00:45:19>,<Venus Aries +25:21:06 +01:12:41>,<Mars Aries +16:27:08 +00:45:19>,<Antes Moon Capricorn +09:16:25 +13:13:02>,<Antes Moon Capricorn +09:16:25 +13:13:02>,<Antes Saturn Capricorn +25:04:15 +00:00:06>,<Antes Mars Virgo +13:32:52 +00:45:19>,<Antes Venus Virgo +04:38:54 +01:12:41>,<Antes Pars Fortuna Virgo +04:31:42 +00:00:00>,<Antes Uranus Virgo +14:53:27 +00:03:14>,<Antes Neptune Libra +22:09:22 +00:02:13>,<Antes Pluto Sagittarius +14:45:25 +00:01:02>,<Antes Chiron Libra +12:23:45 +00:03:43>,<Venus Aries +25:21:06 +01:12:41>,<Jupiter Leo +13:39:12 -00:04:47>,<Jupiter Leo +13:39:12 -00:04:47>,<Mars Aries +16:27:08 +00:45:19>,<Mercury Pisces +00:37:44 +01:29:39>,"[(asc, Asc), (desc, Desc), (mc, MC), (ic, IC), (antes_ruler_desc, Antes Saturn), (antes_ruler_mc..."
1,2016/05/23,12:30,08n07,18e14,<flatlib.chart.Chart object at 0x7f1e28eec580>,<Asc Virgo +27:12:45>,<Desc Pisces +27:12:45>,<MC Gemini +27:30:28>,<IC Sagittarius +27:30:28>,<Pars Fortuna Aries +15:36:29 +00:00:00>,<Sun Gemini +02:47:58 +00:57:39>,<Moon Sagittarius +21:11:41 +12:19:41>,<Saturn Sagittarius +13:54:21 -00:04:19>,<Uranus Aries +22:47:40 +00:02:49>,<Neptune Pisces +11:54:55 +00:00:42>,<Pluto Capricorn +17:11:35 -00:00:57>,<Chiron Pisces +24:43:00 +00:01:47>,<North Node Virgo +18:00:29 -00:03:11>,<South Node Pisces +18:00:29 -00:03:11>,<Pars Spirit Pisces +08:49:02 +00:00:00>,<Pars Glory Capricorn +07:57:50 +00:00:00>,<Pars Crest Virgo +14:43:46 +00:00:00>,<Pars Rock Aries +08:19:08 +00:00:00>,<Mercury Taurus +14:22:23 +00:04:22>,<Jupiter Virgo +13:33:03 +00:02:30>,<Mercury Taurus +14:22:23 +00:04:22>,<Jupiter Virgo +13:33:03 +00:02:30>,<Mars Sagittarius +01:25:22 -00:21:00>,<Antes Moon Capricorn +08:48:19 +12:19:41>,<Antes Mercury Leo +15:37:37 +00:04:22>,<Antes Jupiter Aries +16:26:57 +00:02:30>,<Antes Mercury Leo +15:37:37 +00:04:22>,<Antes Jupiter Aries +16:26:57 +00:02:30>,<Antes Pars Fortuna Virgo +14:23:31 +00:00:00>,<Antes Uranus Virgo +07:12:20 +00:02:49>,<Antes Neptune Libra +18:05:05 +00:00:42>,<Antes Pluto Sagittarius +12:48:25 -00:00:57>,<Antes Chiron Libra +05:17:00 +00:01:47>,<Jupiter Virgo +13:33:03 +00:02:30>,<Saturn Sagittarius +13:54:21 -00:04:19>,<Mercury Taurus +14:22:23 +00:04:22>,<Mars Sagittarius +01:25:22 -00:21:00>,<Mercury Taurus +14:22:23 +00:04:22>,"[(asc, Asc), (desc, Desc), (mc, MC), (ic, IC), (antes_moon, Antes Moon), (antes_pars_fortuna, An..."
2,2018/10/03,00:15,14s42,36w04,<flatlib.chart.Chart object at 0x7f1e28eec5b0>,<Asc Gemini +05:25:08>,<Desc Sagittarius +05:25:08>,<MC Pisces +07:39:39>,<IC Virgo +07:39:39>,<Pars Fortuna Pisces +13:18:26 +00:00:00>,<Sun Libra +09:44:33 +00:59:03>,<Moon Cancer +17:37:52 +14:05:52>,<Saturn Capricorn +03:06:46 +00:

### Calculate aspects

In [56]:
def aspects_calculate(df: pd.DataFrame, row_number: int):
    objects = ObjectsPrepare(df.iloc[row_number]).conbine_class_methods()
    tuple_list = df.transform_id[row_number]

    all_aspects = []

    houses_remove_objs = []
    moon_remove_objs   = ['Asc', 'Desc', 'MC', 'IC', 'Uranus', 'Neptune', 'Pluto', 'Chiron', 'North Node', 'South Node', 'Antes Moon','Antes Uranus', 
                          'Antes Neptune', 'Antes Pluto', 'Antes Chiron']
    main_remove_objs   = ['Asc', 'Desc', 'MC', 'IC', 'Moon', 'Antes Moon']
    houses_aspect_degs = {'Con': 0}
    main_aspect_degs   = {'Con': 0, 'Opp': 180}
    moon_aspect_degs   = AstrologicalConstants.aspekts_degrees
    house_ids          = AstrologicalConstants.houses

    house_objs, moon_objs, main_objs = objects.copy(),objects.copy(), objects.copy() 

    moon_asp_after  = AspectsPrepare.object_aspects('Moon', moon_objs, moon_remove_objs, moon_aspect_degs)
    moon_asp_before = AspectsPrepare.object_aspects('Moon', moon_objs, moon_remove_objs, moon_aspect_degs, before_point_asp='yes', before_orb=1)
    all_aspects.append(moon_asp_after)
    all_aspects.append(moon_asp_before)

    for h_obj in house_ids:
        house_asp_after  = AspectsPrepare.object_aspects(h_obj, house_objs, houses_remove_objs, houses_aspect_degs, before_point_asp='no', after_orb=3)
        house_asp_before = AspectsPrepare.object_aspects(h_obj, house_objs, houses_remove_objs, houses_aspect_degs, before_point_asp='yes', before_orb=3)
        all_aspects.append(house_asp_after)
        all_aspects.append(house_asp_before)

    for m_obj in objects:
        main_asp_after  = AspectsPrepare.object_aspects(m_obj.id, main_objs, main_remove_objs, main_aspect_degs, before_point_asp='no', after_orb=3)
        main_asp_before = AspectsPrepare.object_aspects(m_obj.id, main_objs, main_remove_objs, main_aspect_degs, before_point_asp='yes', before_orb=3)
        all_aspects.append(main_asp_after)
        all_aspects.append(main_asp_before)

    for asp in all_aspects:
        if asp != None and any(var['f_point'] == 'Asc' and var['s_point'] == 'Pluto' for var in asp):
            asc_pluto = asp
        else:  
            asc_pluto = []

    if asc_pluto == []:
        for obj in objects:
            if obj.id == 'Asc' or obj.id == 'Pluto':
                asc_pluto.append(obj) 

        asc_pluto_asp_after = AspectsPrepare.object_aspects('Asc', asc_pluto, houses_remove_objs, houses_aspect_degs, before_point_asp='no', after_orb=20)
        all_aspects.append(asc_pluto_asp_after)

    all_aspects_full = AspectsPrepare.transform_dict_list_type(all_aspects)

    AspectsClearing.remove_duplicate_aspects(all_aspects_full)
    AspectsClearing.remove_antes_unimportant_aspects(all_aspects_full)
    AspectsClearing.remove_node_opposition(all_aspects_full)
    AspectsClearing.remove_antes_with_own_nat_point_aspects(all_aspects_full)
    AspectsClearing.remove_mirror_antes_aspects(all_aspects_full, tuple_list)
    AspectsClearing.remove_mirror_houses_antes_conuction(all_aspects_full)

    for asp in all_aspects_full:
        asp['den_point'] = []

    AspectsPrepare.supplement_houses_and_parts_rulers_aspects(objects, all_aspects_full, tuple_list)
    AspectsPrepare.supplement_conv_stat_aspects(objects, all_aspects_full)
    AspectsPrepare.supplement_moon_complete_denide_weak_aspects(objects, all_aspects_full, tuple_list)
    AspectsPrepare.scale_orb_characteristic(all_aspects_full)
    AspectsPrepare.moon_scale_orb_characteristic(all_aspects_full)
    AspectsPrepare.aspects_rulers_characteristic(tuple_list, all_aspects_full)

    AspectsClearing.remove_houses_one_nodes_conuction(all_aspects_full)  

    return [all_aspects_full]

In [83]:
aspects = []
for x in range(df.shape[0]):
     aspects.append(aspects_calculate(df, x))  

df['aspects'] = aspects        

In [ ]:
# pprint(all_aspects_full, sort_dicts=False)

In [ ]:
# print(len(all_aspects_full))
# for x in all_aspects_full:
#     print(x)

In [ ]:
# for x in objects:
#     if x.id == 'Mars':
#         x.lon = 1
#     if x.id == 'Pluto':
#         x.lon = 2      

In [ ]:
#     Rules for aspekts:
#     Antes Moon - only for house conuction
#     Antes has only house rulers planets, higher planets and Parse Fortuna.

#     Moon aspects by specific points (houses rulers, there antes and fortuna parts with it antes):
#     'moon_conv_clear_compl' - any first aspect by parse fortuna and it antes, or first 'Con' aspect to houses rulers or any last aspect to houses rulers without 
#       any point aspects beefore
#     'moon_conv_compl' - any first aspect by parse fortuna and it antes, or if first 'Con' aspect to houses rulers or any last aspect to houses rulers with 
#       aspected beefore any point among active planets and only 'Con' type for all other points if their not in house rulers
#     'moon_conv_compl_weak' - any first aspect by parse fortuna and it antes or if first 'Con' aspect to houses rulers or any last houses rulers's aspect 
#       which was aspecting beefore parses or other houses rulers and last aspect antes's houses rulers which wasn't aspecting beefore parses or other houses rulers -------
#     'moon_conv' - all other points aspects
#     'moon_conv_denide' - if aspect sing: 'diff' or if aspect's 'tr_orb' > 5.1 
#     'moon_diver', 'moon_diver_weak' - for divergent Moon aspects.

#     Houses rulers and parts rulers aspects are aspecting own parts(houses):
#     'conv_compl', (conv_in_compl', conv_out_compl') - approuch parts(houses) rulers aspecting own parts(houses) without Moon's aspect at part 
#     'conv_denide', (conv_in_weak', conv_out_weak') - if parts(houses) rulers has other early conversation aspect(Moon include) or locate in different sings
#     'diver_weak', (diver_in_weak', diver_out_weak') - if aspect is divergent
#     'diver_denide' - if pars ruler has divergent aspect and it has previos con aspect

#     Other 'conv' and 'stat' aspects:
#     'conv_denide', 'stat_denide' - if points locate in different sings.
#     'conv_weak', 'stat_weak', ('in_conv_weak', 'out_conv_weak', 'in_stat_weak', 'out_stat_weak') - weakened aspects with the other long point 
#     located beetwen points longs. 

#     Remove antes aspects: 
#     If exist aspect between natural points - remove all unimportant aspects with anteses:
#       Stay - ('Mars' con 'Venus') or ('Mars' opp 'Venus')
#       Remove aspects ('Mars' opp 'Antes Venus', 'Mars' opp 'Antes Mars', ' Venus' opp 'Antes Mars', 'Venus' opp 'Antes Venus', 'Antes Mars' opp 'Antes Venus')
#     If exist mirror aspect between natural and antes point without aspect between natural points: 
#       ('Mars' con 'Antes Chiron') and ('Chiron' con 'Antes Mars') 
#       required stay the houses ruler's aspect, if aspect has two houses rulers or any other points - stay any.
#     Antes aspect by own planet without natural points aspect ('Mars' con 'Antes Mars')
#     Remove mirror houses antes aspects: ('Asc' con 'Mars') - ('Dsc' con 'Antes Mars').
#     Remove one mirror 'Moon Node' aspect by houses with 'weak' status from 'den_point' planet or any.